# 파이썬 기반의 AI를 위한 기초수학, 확률및통계

# 정보이론, 엔트로피
- 정보이론 : 로지스틱회귀분석, 모형 해석
- 엔트로피 : 머신러닝, tree 계열 알고리즘, decision tree 등

## 정보이론 information theory
- 정보량 : 어떤 특정 사건이 일어났을 때, 우리는 얼마나 많은 정보를 획득하였는가
  + 확률이 낮을수록 더 놀랍다. ==> 정보량이 크다. ==> 예측이 틀렸다.
- I(x) : 사건(x)가 주는 정보량 (단위; 비트)
- P(x) : 시간 x가 발생할 확률
- 로그 : log (2진법, 비트 기준) 사용
- 예시
    - 내일 비 올 확률 1/2 ==> 1.0 bit
    - 로또 당첨 1/800백만 ==> 23 bit

## 엔트로피
- H(x) : 확률변수 X의 엔트로피 평균 정보량
- P(x) : 각 사건의 확률
- 쉬운 예제 : 앞면, 뒷면 확률 = 0.5
  + 각각 정보량 : log0.5 = 1 bit
  + 엔트로피 : H = 0.5 * 1 + 0.5 * 1 = 1 bit
  + 

In [1]:
import numpy as np

# 각 사건의 확률
probs = [0.5, 0.25, 0.5]

# 각 사건의 정보량
info = [-np.log2(p) for p in probs]

# 평균 정보량 : 엔트로피
entropy = np.sum([p * i for p, i in zip(probs, info)])
entropy

np.float64(1.5)

### 엔트로피의 활용
$$
H = - \sum_{i=1}^{k} p_i \log_2 p_i
$$ 

- 남/여 성별 구분하는 예측 문제
  + 남/여 => 클래스
- 엔트로피 = 0 -> 완전히 순수
- 엔트로피 = 1 -> 섞여 있음

In [1]:
import numpy as np

# 예: 분할된 데이터의 클래스 레이블
labels = ['yes', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'no']  # yes: 4, no: 4 → 완전 섞임

# 클래스별 비율 구하기
values, counts = np.unique(labels, return_counts=True)
probs = counts / counts.sum()

# 엔트로피 계산
entropy = -np.sum(probs * np.log2(probs))

print(f"클래스 분포: {dict(zip(values, counts))}")
print(f"엔트로피: {entropy:.4f} 비트")

클래스 분포: {np.str_('no'): np.int64(4), np.str_('yes'): np.int64(4)}
엔트로피: 1.0000 비트


In [2]:
labels = ['yes', 'yes', 'yes', 'yes', 'no']  # yes: 4, no: 1

# 같은 방식으로 계산
# 클래스별 비율 구하기
values, counts = np.unique(labels, return_counts=True)
probs = counts / counts.sum()

# 엔트로피 계산
entropy = -np.sum(probs * np.log2(probs))

print(f"클래스 분포: {dict(zip(values, counts))}")
print(f"엔트로피: {entropy:.4f} 비트")

클래스 분포: {np.str_('no'): np.int64(1), np.str_('yes'): np.int64(4)}
엔트로피: 0.7219 비트


In [1]:
## 강봉주 
## bonjour.kang@gmail.com
##
## 정보이론
##

In [2]:
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
from pgmpy.factors.discrete import JointProbabilityDistribution as JPD
import pgmpy

pgmpy.__version__

'1.0.0'

In [3]:
# 예제
# 동전 던지기
pk = [1/2, 1/2]
np.round([entropy(pk, base=2), entropy(pk, base=np.e)], 3)

array([1.   , 0.693])

In [4]:
# 주사위 던지기
pk = 1/6*np.ones(6)
np.round([entropy(pk, base=2), entropy(pk, base=np.e)], 3)

array([2.585, 1.792])

In [5]:
# 예제
# 결합 엔트로피
pk = 1/15 * np.array([2, 4, 3, 1, 1, 4])
log_pk = -np.log(pk)
H_XY = np.dot(pk, log_pk)
H_XY.round(3)

np.float64(1.657)

In [6]:
# 예제
# 조건부 엔트로피

# 결합 확률 정의
prob = 1/15 * np.array([2, 4, 3, 1, 1, 4])
fxy = JPD(['X', 'Y'],[2, 3], prob)
print(fxy)

+------+------+----------+
| X    | Y    |   P(X,Y) |
+======+======+==========+
| X(0) | Y(0) |   0.1333 |
+------+------+----------+
| X(0) | Y(1) |   0.2667 |
+------+------+----------+
| X(0) | Y(2) |   0.2000 |
+------+------+----------+
| X(1) | Y(0) |   0.0667 |
+------+------+----------+
| X(1) | Y(1) |   0.0667 |
+------+------+----------+
| X(1) | Y(2) |   0.2667 |
+------+------+----------+


In [7]:
# 조건부 확률 예시
prob = fxy.conditional_distribution([('Y', 0)], inplace=False)
print(prob)

+------+--------+
| X    |   P(X) |
+======+========+
| X(0) | 0.6667 |
+------+--------+
| X(1) | 0.3333 |
+------+--------+


In [8]:
# 조건부 엔트로피 계산
res_sum = []

# 모든 x에 대하여
for i in fxy.state_names['X']:
    # 모든 y에 대하여
    for j in fxy.state_names['Y']:
        cond_y = fxy.conditional_distribution([('Y', j)], inplace=False).values[i]
        res = -fxy.values[i,j] * np.log(cond_y)
        res_sum.append(res)
# 결과
np.sum(res_sum).round(3)

np.float64(0.613)

In [9]:
# H(X, Y) - H(Y) 이용
# H(X, Y) 계산
H_XY = np.dot(fxy.values.reshape(-1) ,-np.log(fxy.values.reshape(-1)))

# H_Y 계산
prob_y = fxy.marginal_distribution(['Y'], inplace=False).values
H_Y = np.dot(prob_y ,-np.log(prob_y))

# 결과 계산
np.round(H_XY-H_Y, 3)

np.float64(0.613)

In [10]:
# 예제
# 상호 정보 계산

# 결합 확률 정의
pxy = 1/15 * np.array([2, 4, 3, 1, 1, 4])
fxy = JPD(['X', 'Y'],[2, 3], pxy)

# 주변확률 계산
px = fxy.marginal_distribution(['X'], inplace=False).values
py = fxy.marginal_distribution(['Y'], inplace=False).values

# 상호 정보 계산
h_xy = np.dot(pxy, -np.log(pxy))
h_x = np.dot(px, -np.log(px))
h_y = np.dot(py, -np.log(py))

# 결과
i_xy = h_x + h_y - h_xy
i_xy.round(3)

np.float64(0.06)

In [11]:
# 예제
# 교차 엔트로피 계산
px = [1/4, 1/2, 1/4]
qx = [1/3, 1/3, 1/3]

hpq = np.dot(px, -np.log(qx))
hpq.round(3)

np.float64(1.099)